# 🌊 Make Continuum Land Maps 
<br>
<img style="float: left; padding-right: 15px; padding-left: 0px;" src="../sources/images/logo_continuum.png" width="260px" align=”left” >

<div style="text-align: justify">This is a Jupyter Notebook, a web-based interactive development environment that allows to create and share python codes.
This notebook loads parameters from a configuration file (`config.json`), saved in the `settings` folder and do a set of operations for the preparations of the static land data for Continuum model:

- performs Digital Elevation Model (DEM) preprocessing
- hydrological condition the DEM with provided spatial datasets
- compute the key hydrological derivatives
- provide the full set of hydrological model land data

The entire workflow is documented with clear explanations and visual plots to help you understand each step.



## 🔧 Preliminary Setup

This section handles the initial configuration, including setting the project file, importing necessary libraries, and preparing the working environment.

#### Specify Configuration File

This cell defines the name of the configuration file to be used for the entire workflow. The configuration file, `awash.yaml` in this case, contains all the parameters for the project.

In [2]:
# Set the name of the settings file
settings_file = "awash.json"

print(f"Using settings file: {settings_file}")

Using settings file: awash.json


#### Import Required Libraries

This cell imports all the Python libraries necessary to run the notebook. These libraries cover a wide range of functionalities, including data handling (`json`, `os`, `pathlib`, `numpy`), geospatial operations (`rasterio`, `geopandas`, `pygrass`), plotting (`matplotlib`), and custom tools from local modules.

In [ ]:
import json
import os
from pathlib import Path

import os, math, time, logging, shutil, subprocess
import numpy as np
import math
import rasterio as rio
import geopandas as gpd
import matplotlib.pyplot as plt
from grass_session import Session
from grass.pygrass.modules.shortcuts import general as g, raster as r, vector as v
from grass.pygrass.modules import Module
from grass.script import parse_key_val
import grass.script as gs
from subprocess import PIPE
from rasterio import Affine, features
from copy import deepcopy

import geo_tools, plot_tools, io_tools, hmc_tools

from IPython.display import display, clear_output
%matplotlib ipympl

print("All required libraries have been imported.")

All required libraries have been imported.


#### Set Up Working Paths and Generate Folder Structure

This cell sets the project's root directory and loads the configuration from the specified YAML file. It then uses the `path` settings from the configuration to create the necessary folder structure for the project if it doesn't already exist.

In [4]:
# Define the project root directory
project_root = str(Path().cwd()).replace('notebook','')

# Load configuration from the YAML file
with open(os.path.join(project_root, "settings", settings_file), 'r') as f:
    config = json.load(f)
    
print(f"Configuration loaded from settings file: {settings_file}")

# Generate folder tree based on the configuration
path_settings = config['path']
for key in path_settings.keys():
    path_settings[key] = os.path.join(project_root, "projects", config['general']['project'], path_settings[key])
    os.makedirs(path_settings[key], exist_ok=True)
    
print("Folder tree generated successfully.")
print(f"Project root: {project_root}")

Configuration loaded from settings file: awash.json
Folder tree generated successfully.
Project root: /home/continuumuser/workdir/


## 🛑 **WARNING! Copy all required data to the `data` folder before proceeding** 🛑
In particular the DEM and, if present, rivers, depression and walls

---

## ⛰️ DEM Preprocessing

This section focuses on preparing the Digital Elevation Model (DEM) for hydrological analysis. This involves reading the original DEM, resampling its resolution if needed, and filling sinks.

#### Read Input Digital Elevation Model

This cell reads the original DEM file specified in the configuration. It handles nodata values, replacing them with `NaN` for proper numerical operations. The cell then calculates and prints the DEM's resolution and plots the original DEM for visual inspection.

In [5]:
# Open the original DEM file
with rio.open(os.path.join(path_settings['data'], config['data']['dem'])) as src:
    dem_orig = src.read(1)
    tr = src.transform
    profile = src.profile
    nod = src.nodata if src.nodata is not None else -9999
    dem_orig[dem_orig==nod] = np.nan
    
print("Original DEM read successfully.")
print(f"DEM dimensions: {dem_orig.shape}")
    
# Plot the original DEM
plot_tools.single_plot(dem_orig, "Original DEM", cmap='terrain')

# Calculate and print the resolution of the input DEM
working_res_km = geo_tools.deg2km((np.abs(tr[0]) + np.abs(tr[4])) / 2)
print(f"Input DEM resolution is around {working_res_km:.3f} km.")

Original DEM read successfully.
DEM dimensions: (16615, 21766)


<IPython.core.display.Javascript object>

Input DEM resolution is around 0.093 km.


#### Resample DEM Resolution

This cell checks a flag in the configuration to determine if the DEM needs to be resampled to a different resolution. If the flag is set to `True`, it resamples the DEM to the specified output resolution with cubic interpolation, saves the new DEM, and plots it for comparison. Otherwise, it uses the original DEM.

In [6]:
# Check if DEM resampling is required
if config['flags']['change_dem_resolution']:
    if config['settings']['out_res_km'] is None:
        raise ValueError("Please provide a numerical value for out_res_km in the configuration.")
        
    working_res_km = config['settings']['out_res_km']
    print(f"Resampling DEM to {working_res_km} km resolution...")
    
    # Resample the DEM
    dem_file_raw, profile = geo_tools.resample(out = os.path.join(path_settings['ancillary'], "dem_resampled.tif"),
                                  inp = os.path.join(path_settings['data'], config['data']['dem']),
                                  target_res_deg = geo_tools.km2deg(working_res_km))
                                  
    print("Resampling complete. Plotting resampled DEM...")
    
    # Read and plot the resampled DEM
    with rio.open(os.path.join(path_settings['ancillary'], "dem_resampled.tif")) as src:
        dem_working = src.read(1)
        tr = src.transform
        nod = src.nodata if src.nodata is not None else -9999
        dem_working[dem_working==nod] = np.nan
    
    plot_tools.single_plot(dem_working, "Resampled DEM", cmap='terrain')
else:
    print("No resampling of the original DEM required.")
    dem_working = dem_orig
    dem_file_raw = os.path.join(path_settings['data'], config['data']['dem'])

Resampling DEM to 1 km resolution...
Resampling complete. Plotting resampled DEM...


<IPython.core.display.Javascript object>

#### Fill Sinks with SAGA GIS

This cell uses SAGA GIS via command-line calls to process the DEM. It first identifies and fills sinks, which are low points in the DEM that trap water flow. This step is crucial for accurate hydrological analysis. The temporary SAGA files are deleted afterward.

In [7]:
# Define file paths for SAGA GIS processing
saga_dem = os.path.join(path_settings['ancillary'], "dem_saga.sdat")
sink_route = os.path.join(path_settings['ancillary'], "sinkroute_saga.sdat")
dem_file_processed = os.path.join(path_settings['ancillary'], "dem_preprocessed.tif")

print("Starting DEM sink-filling process using SAGA GIS.")

# Import DEM to SAGA format
display("Step 1/4: Importing DEM to SAGA format...")
os.system(f"gdal_translate -q -of SAGA {dem_file_raw} {saga_dem}")

# Identify sink routes
display("Step 2/4: Identifying sink routes...")
os.system(f"saga_cmd -f=s ta_preprocessor 1 -ELEVATION {saga_dem} -SINKROUTE {sink_route}")

# Fill sinks
display("Step 3/4: Filling sinks...")
os.system(f"saga_cmd -f=s ta_preprocessor 2 -DEM {saga_dem} -SINKROUTE {sink_route} -DEM_PREPROC {saga_dem}")

# Convert processed DEM back to GeoTIFF
display("Step 4/4: Converting processed DEM to GeoTIFF...")
os.system(f"gdal_translate -of GTiff {saga_dem} {dem_file_processed} -co COMPRESS=DEFLATE -ot Float32")

# Read the processed DEM for further use
with rio.open(dem_file_processed) as src:
    dem_working = src.read(1)
    tr = src.transform
    nod = src.nodata if src.nodata is not None else -9999
    dem_working[dem_working==nod] = np.nan
    
print("DEM sink-filling process complete.")

# Clean up temporary SAGA files
print("Cleaning up temporary SAGA files.")
tmp = Path(path_settings['ancillary'])
for f in tmp.glob("*saga*"):
    f.unlink()
print("\n✅ DEM preprocessing succesful!")

Starting DEM sink-filling process using SAGA GIS.


'Step 1/4: Importing DEM to SAGA format...'

'Step 2/4: Identifying sink routes...'

'Step 3/4: Filling sinks...'

'Step 4/4: Converting processed DEM to GeoTIFF...'

Input file size is 2016, 1539
0...10...20...30...40...50...60...70...80...90...100 - done.
DEM sink-filling process complete.
Cleaning up temporary SAGA files.

✅ DEM preprocessing succesful!


---

## 🪏 Hydrological Conditioning

This section applies various conditioning methods to the DEM, such as carving river networks, using depressions for endorheic basins, and adding walls, to better represent the real-world hydrology.

#### Carve DEM

This cell checks a flag to see if the DEM should be carved using a provided stream network shapefile. If enabled, it rasterizes the shapefile and subtracts a fixed value from the DEM along the stream lines, ensuring that water flows along the defined network.

In [8]:
dem_carved = deepcopy(dem_working)
carve_depth = 3 #meters

if config['flags']['carve_dem']:
    if config['data']['stream'] is None:
        raise ValueError("Please provide a stream shapefile for carving in the configuration.")
        
    print(f"Starting DEM carving with {config['data']['stream']} shapefile (this may take a while)...", flush=True)
    
    # Read the stream shapefile
    gdf = gpd.read_file(os.path.join(path_settings['data'], config['data']['stream']))
    shapes = ((geom, 1) for geom in gdf.geometry)
    
    print("Rasterizing shapefile...", flush=True)
    # Rasterize the stream shapefile to create a mask
    stream_mask = features.rasterize(
                shapes,
                out_shape=dem_working.shape,
                transform=tr,
                fill=0,
                dtype='uint8').astype(bool)
                
    # Carve the DEM by subtracting a value along the stream mask
    dem_carved = dem_working - stream_mask * carve_depth
    
    print("\n ✅DEM carving complete.")
    
    # Plot the carving network and compare the original and carved DEMs
    plot_tools.single_plot(stream_mask, "Carving Network", cmap='terrain')
    plot_tools.locked_plot(dem_working, dem_carved, title1="Resampled DEM", title2="Carved DEM")
else:
    print("DEM carving is not active. Skipping this step.")

DEM carving is not active. Skipping this step.


#### Use Depressions for Endorheic Basins

This cell checks a flag to see if depression points should be used to create holes in the DEM. This is useful for modeling endorheic basins, which are closed drainage basins that retain water and do not drain to an outlet.

In [9]:
dem_depressed = deepcopy(dem_carved)

if config['flags']['use_depressions']:
    if config['data']['depressions'] is None:
        raise ValueError("Please provide a point depression shapefile in the configuration.")
        
    print(f"Creating holes in the DEM with {config['data']['depressions']} shapefile...", flush=True)
    
    # Read the depressions shapefile
    gdf = gpd.read_file(os.path.join(path_settings['data'], config['data']['depressions']))
    shapes = ((geom, 1) for geom in gdf.geometry)
    
    print("Rasterizing shapefile...", flush=True)
    # Rasterize the depression shapefile to create a mask
    depression_mask = features.rasterize(
                shapes,
                out_shape=dem_working.shape,
                transform=tr,
                fill=0,
                dtype='uint8'
            ).astype(bool)
            
    # Set the DEM values at depression points to NaN
    dem_depressed[depression_mask] = np.nan
    
    print("\n✅ Depression operations finished.")
else:
    print("Using depressions is not active. Skipping this step.")

Using depressions is not active. Skipping this step.


#### Use Walls

This cell checks a flag to see if walls should be added to the DEM. Walls are used to define boundaries for water flow, which can be useful for modeling specific catchment areas or preventing water from flowing into certain regions.

In [10]:
dem_walled = deepcopy(dem_depressed)
wall_heigth = 30 #meters

# Save the DEM before adding walls
dem_file_unwalled = os.path.join(path_settings['ancillary'], "dem_unwalled.tif")
with rio.open(dem_file_unwalled, 'w', **profile) as dst:
    dst.write(dem_walled[np.newaxis, :, :])
    
if config['flags']['use_walls']:
    if config['data']['wall'] is None:
        raise ValueError("Please provide a wall line shapefile in the configuration.")
        
    print(f"Walling the DEM with {config['data']['wall']} shapefile (this may take a while)...", flush=True)
    
    # Read the walls shapefile
    gdf = gpd.read_file(os.path.join(path_settings['data'], config['data']['wall']))
    shapes = ((geom, 1) for geom in gdf.geometry)
    
    print("Rasterizing shapefile...", flush=True)
    # Rasterize the walls shapefile to create a mask
    walls_mask = features.rasterize(
                shapes,
                out_shape=dem_working.shape,
                transform=tr,
                fill=0,
                dtype='uint8'
            ).astype(bool)
            
    # Add a fixed value to the DEM along the wall lines
    dem_walled = dem_walled + walls_mask * wall_heigth
    
    #print("\n✅ Walling operation finished.")
    
    # Plot the walls mask
    plot_tools.single_plot(walls_mask, "Walls", cmap='terrain')
else:
    print("Using walls is not active. Skipping this step.")

Using walls is not active. Skipping this step.


#### Write Conditioned DEM

This cell saves the final, conditioned DEM to a GeoTIFF file. This DEM is now ready for use in the hydrological derivatives calculation. The temporary DEM arrays are deleted to free up memory.

In [11]:
dem_file_conditioned = os.path.join(path_settings['ancillary'], "dem_conditioned.tif")
with rio.open(dem_file_conditioned, 'w', **profile) as dst:
    dst.write(dem_walled[np.newaxis, :, :])
    
print(f"Conditioned DEM saved to: {dem_file_conditioned}")

# Clean up temporary DEM arrays from memory
del dem_walled, dem_carved, dem_depressed
print("Temporary DEM arrays deleted.")

Conditioned DEM saved to: /home/continuumuser/workdir/projects/training_eth/ancillary/dem_conditioned.tif
Temporary DEM arrays deleted.


---

## 🌐 Compute land maps

This section uses GRASS GIS to calculate key hydrological derivatives from the conditioned DE and the other land maps

#### Start GRASS Session

This cell initializes a GRASS GIS session. It sets up the database, location, and mapset. If the mapset doesn't exist, it is created. This session provides the environment for all subsequent GRASS GIS commands.

In [12]:
gisdb = "/home/continuumuser/grassdata"
location = "WGS84"
mapset = f"{config['general']['domain']}_land_data"   # es. "awash_land_data"

location_path = os.path.join(gisdb, location)
perm_path     = os.path.join(location_path, "PERMANENT")
mapset_path   = os.path.join(location_path, mapset)
lockfile      = os.path.join(mapset_path, ".gislock")

# Create location if missing
if not os.path.isdir(perm_path):
    print(f"Creating GRASS LOCATION {location} (EPSG:4326)...")
    subprocess.run(["grass", "--text", "-c", "EPSG:4326", perm_path], check=True)
else:
    print(f"LOCATION {location} already exists.")

# Clean lock files
if os.path.exists(lockfile):
    try:
        os.remove(lockfile)
        print(f"Removed stale lock: {lockfile}")
    except Exception as e:
        print(f"Warning: cannot remove lock {lockfile}: {e}")

if os.path.isdir(mapset_path):
    print(f"Removing existing MAPSET: {mapset_path}")
    shutil.rmtree(mapset_path)

print(f"Creating new MAPSET: {mapset_path}")
os.makedirs(mapset_path, exist_ok=True)

# Coopy required files from PERMANENT
for fname in ("WIND", "DEFAULT_WIND", "PROJ_INFO", "PROJ_UNITS"):
    src = os.path.join(perm_path, fname)
    dst = os.path.join(mapset_path, fname)
    if os.path.exists(src):
        shutil.copy(src, dst)

# Explicitly export GISRC
gisrc_path = f"/tmp/grassrc_{location}_{mapset}"
with open(gisrc_path, "w") as f:
    f.write(f"GISDBASE: {gisdb}\n")
    f.write(f"LOCATION_NAME: {location}\n")
    f.write(f"MAPSET: {mapset}\n")

os.environ["GISRC"] = gisrc_path
print("GISRC set to:", os.environ["GISRC"])

# 5) Registra i Module PyGRASS che usi nel notebook
r.in_gdal        = Module("r.in.gdal")
r.out_gdal       = Module("r.out.gdal")
r.stream_basins  = Module("r.stream.basins")  # NB: il modulo si chiama r.stream.basins
v.in_ogr         = Module("v.in.ogr")

LOCATION WGS84 already exists.
Creating new MAPSET: /home/continuumuser/grassdata/WGS84/awash_land_data
GISRC set to: /tmp/grassrc_WGS84_awash_land_data


#### Import DEM into GRASS and Compute Hydroderivatives

This cell imports the preprocessed and conditioned DEMs into the GRASS GIS session. It then uses the `r.watershed` module to compute several hydrological derivatives, including basins, streams, total contributing area (TCA), and drainage direction.

In [13]:
print("Importing DEMs into GRASS GIS and computing hydroderivatives.")
# Import the processed and conditioned DEMs
r.in_gdal(input=dem_file_processed, output="dem_clean", flags='o', overwrite=True, quiet=True)
r.in_gdal(input=dem_file_conditioned, output="dem_conditioned", flags='o', overwrite=True, quiet=True)#g.region(raster="dem_clean", save='extended_region', overwrite=True)

# Set region in grass 8
gs.run_command("g.region", raster="dem_clean", flags="a")
gs.run_command("g.region", save="extended_region", overwrite=True)
print("Settings model region...")
g.region(flags="p")

r.mapcalc(f"dem_clean = if(isnull(dem_conditioned), null(), dem_clean)", overwrite=True)
print("DEMs imported into GRASS GIS.")

# Calculate hydroderivatives using r.watershed
print("Calculating hydroderivatives (basins, streams, TCA, and drainage direction)...")
r.watershed(elevation = "dem_conditioned",
            threshold = int(config["settings"]["soglia_basin_km2"] / (working_res_km ** 2)),
            basin="basin",
            stream="stream",
            accumulation="TCA",
            drainage="pnt_grass",
            flags="sab", overwrite=True, quiet=True)

print("\n✅ Hydroderivatives calculation complete.")

Importing DEMs into GRASS GIS and computing hydroderivatives.
Settings model region...
projection: 3 (Latitude-Longitude)
zone:       0
datum:      wgs84
ellipsoid:  wgs84
north:      16:06:46.500226N
south:      2:16:20.485746N
west:       31:33:31.5E
east:       49:41:20.665167E
nsres:      0:00:32.375578
ewres:      0:00:32.375578
rows:       1539
cols:       2016
cells:      3102624
DEMs imported into GRASS GIS.
Calculating hydroderivatives (basins, streams, TCA, and drainage direction)...

✅ Hydroderivatives calculation complete.


#### Activate mask

This cell checks a flag to see if a mask should be applied to the domain. If enabled, it imports a mask shapefile into GRASS GIS, buffers it, and uses it to define the active computational region. This is useful for focusing the analysis on a specific area of interest.

In [14]:
if config['flags']['use_mask']:
    if config['data']['mask'] is None:
        raise ValueError("Please provide a mask shapefile in the configuration.")
        
    print(f"Applying mask to the domain with {config['data']['mask']} shapefile.", flush=True)
    
    # Remove any existing mask
    try: 
        r.mask(flags='r')
    except: 
        pass
        
    # Import the mask shapefile as a GRASS vector map
    v.in_ogr(input=os.path.join(path_settings['data'], config['data']['mask']), output='mask_v', overwrite=True, quiet=True)
    
    # Buffer the vector mask and set it as the GRASS region
    v.buffer(input='mask_v', output='mask_v_buffer', distance=geo_tools.km2deg(working_res_km) * (3/4), quiet=True, overwrite=True)
    
    # Apply the mask
    r.mask(vector="mask_v", quiet=True)
    gs.run_command("g.region", vector="mask_v_buffer",  align="pnt_grass", flags="a")
    gs.run_command("g.region", save="masked_region", overwrite=True)
    
    print("\n✅ Mask applied successfully.")
else:
    print("Masking is not active. Skipping this step.")

Masking is not active. Skipping this step.


#### Write DEM map

This cell generate the map of the:
* **dem** ➡️ the cleaned Digital Elevation Model (m) including only the depressions (if used)

In [15]:
# Define output file paths for the DEM map
dem_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".dem.txt")
temp_map = os.path.join(path_settings["ancillary"], "temp.tiff")

hydroderivatives = {}

print("Exporting DEM map...")

# Export the 'dem_clean' raster from GRASS GIS
r.out_gdal(input='dem_clean', output=temp_map, format='GTiff', overwrite=True, type='Float32', nodata=-9999, flags='fc', quiet=True)

# Convert the GeoTIFF to an ASCII Grid
io_tools.convertAIIGrid(temp_map, dem_map, 'Float32')

# Read the exported DEM into the hydroderivatives dictionary
var = "dem"
with rio.open(dem_map) as src:
    hydroderivatives[var] = src.read(1).astype("float32")
    nod = -9999
    hydroderivatives[var][hydroderivatives[var]==nod] = np.nan
    
print("\n✅ DEM map exported successfully.")

Exporting DEM map...

✅ DEM map exported successfully.


#### Write area and choice maps

This cell exports:
* **area** ➡️ the total contributing area in number of cells
* **choice** ➡️ the rasterized river network maps 

Both are plotted for visual inspection.

In [16]:
# Define output file paths for area and choice maps
area_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".area.txt")
choice_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".choice.txt")

# Compute and export upstream area (TCA)
print('Exporting upstream area ... ', flush=True)
r.out_gdal(input="TCA", output=temp_map, format='GTiff', overwrite=True, nodata=-9999, type='Int32', flags='fc', 
           quiet=True)
io_tools.convertAIIGrid(temp_map, area_map, 'Int32')

# Compute and export choice map (river network)
print('Exporting choice map ... ', flush=True)
r.mapcalc("choice = if(isnull(stream), 0, 1)", overwrite=True)
r.out_gdal(input='choice', output=temp_map, format='GTiff', overwrite=True, nodata=-9999, type='Int32', flags='fc', 
           quiet=True)
io_tools.convertAIIGrid(temp_map, choice_map, 'Int32')

# Read the exported maps into the hydroderivatives dictionary
for var in ["area", "choice"]:
    path = globals()[f"{var}_map"]
    with rio.open(path) as src:
        hydroderivatives[var] = src.read(1).astype("float32")
        nod = -9999
        hydroderivatives[var][hydroderivatives[var]==nod] = np.nan
print("\n✅ Upstream area and choice maps exported successfully.", flush = True)    
# Plot the maps
plot_tools.locked_plot(hydroderivatives["choice"], hydroderivatives["area"], "Network", "Area (n cells)", 
                       share_palette=False)

Exporting upstream area ... 


         Int32. This can be avoided by using Float64.


Exporting choice map ... 

✅ Upstream area and choice maps exported successfully.


<IPython.core.display.Javascript object>

#### Write the pnt map

This cell exports the:
* **pnt** ➡️ the drainage direction map, that indicates for each cell in what direction the water is flowing following the D8 convenction and the codification of Continuum. 

It also export the ancillary map:

* **drainage_direction** ➡️ the drainage direction map with QGIS convenction that can be used for basin delineation in QGIS

In [17]:
# Define output file paths for drainage direction maps
pnt_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".pnt.txt")
pntQgis_map = os.path.join(path_settings["ancillary"], "drainage_direction.tif")

print("Starting conversion and export of drainage direction map...")

# Write GRASS to HMC conversion rules file
hmc_tools.writeGrass2HMC(path_settings['ancillary'])

# Convert drainage direction to Continuum standards
display('Convert drainage direction to Continuum standards ... ')
r.reclass(input="pnt_grass",
          output="pnt_HMC", rules=os.path.join(path_settings['ancillary'], "grass2cont.txt"),
          overwrite=True, quiet=True)
          
# Export both the original GRASS and the converted HMC drainage maps
r.out_gdal(input='pnt_grass', output=pntQgis_map, format='GTiff', overwrite=True, type='Int16', flags='fc', quiet=True)
r.out_gdal(input='pnt_HMC', output=temp_map, format='GTiff', overwrite=True, type='Int16', flags='fc', quiet=True)
io_tools.convertAIIGrid(temp_map, pnt_map, 'Int32')

# Read the exported drainage direction map into the hydroderivatives dictionary
var = "pnt"
with rio.open(pnt_map) as src:
    hydroderivatives[var] = src.read(1).astype("float32")
    nod = -9999
    hydroderivatives[var][hydroderivatives[var]==nod] = np.nan
    
print("Drainage direction map exported successfully.")

# Plot the drainage direction map with a D8 legend
plot_tools.single_plot(hydroderivatives["pnt"], "Drainage Direction", cmap='viridis')
ax = plt.gca()
plot_tools.add_d8_legend(ax, hydroderivatives["pnt"], cmap_name="gist_rainbow", ncol=4, pad=0.15)

Starting conversion and export of drainage direction map...


'Convert drainage direction to Continuum standards ... '

Drainage direction map exported successfully.


<IPython.core.display.Javascript object>

/home/continuumuser/workdir/sources/libraries/plot_tools.py:181: RuntimeWarning: invalid value encountered in cast
  im = ax.imshow(data.astype(int), cmap=cmap, norm=norm)


#### Compute latitude, longitude and areacell maps 

This cell uses GRASS GIS to calculate the Continuum georeferencing static maps, in the details: 
* **latitude** ➡️ Latitude in degrees (WGS84)
* **longitude** ➡️ Longitude in degrees (WGS84)
* **areacell** ➡️ Area of each cell in m^2 

It then exports these maps to GeoTIFF format and converts them to the ASCII Grid format, which is the format suitable for Continuum.

In [18]:
# Define output file paths for lat, lon, and areacell maps
temp_map = os.path.join(path_settings["ancillary"], "temp.tiff")
lat_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".lat.txt")
lon_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".lon.txt")
areacell_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".areacell.txt")

hydroderivatives = {}

# Remove any existing mask
try: 
    r.mask(flags='r')
except: 
    pass
    
# Calculate latitude and longitude maps using GRASS mapcalc
r.mapcalc("lat = y()", overwrite=True, quiet=True)
r.mapcalc("lon = x()", overwrite=True, quiet=True)
r.mapcalc("areacell = area()", overwrite=True, quiet=True)

# Export maps to GeoTIFF and then to ASCII Grid format
print("Exporting latitude, longitude, and areacell maps...")
for var in ["lat", "lon", "areacell"]:
    path = globals()[f"{var}_map"]
    r.out_gdal(input=var, output=temp_map, format='GTiff', overwrite=True, type='Float32', flags='fc', quiet=True)
    io_tools.convertAIIGrid(temp_map, path, 'Float32')
    with rio.open(path) as src:
        hydroderivatives[var] = src.read(1).astype("float32")
        nod = -9999
        hydroderivatives[var][hydroderivatives[var]==nod] = np.nan
        
print("\n✅ Latitude, longitude, and areacell maps are ready.")

ERROR: No existing MASK to remove


Exporting latitude, longitude, and areacell maps...


         raster <lat>. This can be avoided by using Float64
         raster <lon>. This can be avoided by using Float64
         raster <areacell>. This can be avoided by using Float64



✅ Latitude, longitude, and areacell maps are ready.


#### Write alpha and beta maps

This final section calculates the maps of the model related to the slope, in particular:
* **alpha** ➡️ represent the downslope index and is used for modelling the behaviour of the water table
* **beta** ➡️  correspond to alpha corrected in the channel cells with a smoothed local slope for modelling the discharge in channels

The calculation is performed by a custom tool and the results are exported as ASCII Grids and then plotted.

In [19]:
alpha_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".alpha.txt")
beta_map = os.path.join(path_settings["output"], config["general"]["domain"] + ".beta.txt")
temp_map = os.path.join(path_settings["ancillary"], "temp.tiff")

# Apply the mask
if config['flags']['use_mask']:
    r.mask(vector="mask_v", quiet=True)
    gs.run_command("g.region", region="masked_region")
    
print('Start computing alpha and beta (it might take a while)')
# Alpha & Beta
hmc_tools.makeAlphaBeta(path_settings["output"] + "/", path_settings['ancillary'] + "/", config["general"]["domain"])
for var in ['alpha', 'beta']:
    path = globals()[f"{var}_map"]
    out_fn = os.path.join(path_settings['ancillary'], f"temp_{var}.tif")
    r.in_gdal(input=out_fn, output=var, flags='o', overwrite=True, quiet=True)
    r.out_gdal(input=var, output=temp_map, format='GTiff', overwrite=True, type='Float32', flags='fc', quiet=True)
    io_tools.convertAIIGrid(temp_map, path, 'Float32', precision=6)
    with rio.open(path) as src:
        hydroderivatives[var] = src.read(1).astype("float32")
        nod = -9999
        hydroderivatives[var][hydroderivatives[var]==nod] = np.nan
        
plot_tools.locked_plot(hydroderivatives["alpha"], hydroderivatives["beta"], "Alpha", "Beta", 
                       cmap1 = "viridis", cmap2 = "viridis", share_palette=True)
print("\n✅ Alpha and beta maps exported successfully.")


Start computing alpha and beta (it might take a while)
1/3 Computing alpha ...
2/3 Computing beta ...
3/3 Writing alpha and beta ...


<IPython.core.display.Javascript object>


✅ Alpha and beta maps exported successfully.


#### Write ancillary Maps

This section generates additional maps that are useful for understanding the hydrological properties of the domain and for further analysis:
* **mbsn** ➡️ The maps of the macrobasins to be used for identifying useful basins masks
* **upstream_area_km2** ➡️ The maps of the upstream area in km2

In [20]:
# Macrobasins map
gs.run_command("g.region", region="extended_region")
mbsn_map = os.path.join(path_settings["ancillary"], "mbsn.tif")
areaKm_map = os.path.join(path_settings["ancillary"], "upstream_area_km2.tif")

r.stream_basins(direction=f"pnt_grass",
                stream_rast=f"stream",
                flags='l', basins="mbsn", overwrite=True, quiet=True)

r.out_gdal(input="mbsn", output=mbsn_map, format='GTiff', overwrite=True, type='Int32')
with rio.open(mbsn_map) as src:
    mbsn = src.read(1)

# Cell area and km²
r.mapcalc("areaCellkm = areacell/(10^6)", overwrite=True, quiet=True)
r.accumulate(direction="pnt_grass", format='auto', accumulation="temp.TCA.km2", weight='areaCellkm', overwrite=True, quiet=True)
out_km2 = os.path.join(path_settings['ancillary'], "upstream_area_km2.tif")
r.out_gdal(input="temp.TCA.km2", output=areaKm_map, format='GTiff', overwrite=True, type='Float32')
with rio.open(areaKm_map) as src:
    areaKm = src.read(1)
    
plot_tools.locked_plot(mbsn, areaKm, "Macrobasins", "Upsteream area (km2)", 
                       cmap1 = "rainbow", cmap2 = "viridis", share_palette=False)

         raster <temp.TCA.km2>. This can be avoided by using Float64


<IPython.core.display.Javascript object>

#### Clean temporary files

In [21]:
# Clean up temporary files
print("Cleaning up temporary files.")
tmp = Path(path_settings['ancillary'])
for f in tmp.glob("temp*"):
    f.unlink()
print("\n✅ Temporary files cleaned!")

Cleaning up temporary files.

✅ Temporary files cleaned!


## ✅  **CONGRATULATION! All static land data for the model are ready** ✅ 